In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pyspiel
import open_spiel.python.games
from open_spiel.python.observation import make_observation
from open_spiel.python.examples.ubc_decorators import TakeSingleActionDecorator
from open_spiel.python.examples.straightforward_agent import StraightforwardAgent
from open_spiel.python.examples.ppo_utils import EnvParams
from auctions.webutils import *
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


/home/ubuntu/.pyenv/versions/3.8.2/envs/venv/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):
/home/ubuntu/.pyenv/versions/3.8.2/envs/venv/lib/python3.8/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/home/ubuntu/.pyenv/versions/3.8.2/envs/venv/lib/python3.8/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


In [10]:
game = pyspiel.load_game('python_clock_auction', dict(filename='sats_2regions_2licenses_2players_5types.json'))

In [21]:
np.where((game.auction_params.all_bids == [2,2]).all(axis=1))[0][0]

8

In [3]:
straightforward = TakeSingleActionDecorator(StraightforwardAgent(1, game), game.num_distinct_actions())

In [4]:
state = game.new_initial_state().child(0).child(0).child(6).child(3)

In [9]:
dir(state)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_action_to_string',
 '_apply_action',
 '_auction_finished',
 '_cur_player',
 '_determine_tiebreaks',
 '_final_payments',
 '_game_over',
 '_handle_bids',
 '_is_chance',
 '_legal_actions',
 '_post_process',
 '_process_bids',
 'action_to_string',
 'aggregate_demand',
 'apply_action',
 'apply_action_with_legality_check',
 'apply_actions',
 'apply_actions_with_legality_checks',
 'auction_params',
 'bidders',
 'chance_outcomes',
 'child',
 'clock_prices',
 'clone',
 'current_player',
 'distribution_support',
 'full_history',
 'get_allocation',
 'get_final_payments',
 'get_game',
 'get_type',
 'history',
 'his

In [6]:
price = np.array(state.sor_prices[-1])
prices = np.array([price @ bid for bid in state.auction_params.all_bids])

state.bidders[0].bidder.get_profits(prices)

array([  0., 111.,  95., 419., 530., 514., 648., 759., 744.])

In [7]:
state.legal_actions()

[1, 3, 6]

In [8]:
# Load checkpoint
seed = 1234
equilibrium_solver_run_checkpoint = get_checkpoint_by_name('ppo_test_15', 'sats_2regions_2licenses_2players_5types-aug11ppo-100')
env_params = EnvParams(track_stats=True, seed=seed, num_envs = 1, history_prefix=[0, 0], sync=True)
env_and_policy = ppo_db_checkpoint_loader(equilibrium_solver_run_checkpoint, env_params=env_params)
env = env_and_policy.env

In [10]:
env_and_policy.agents[1] = straightforward

In [11]:
agents = env_and_policy.agents
time_step = env.reset()
episode_counter = len(env)
num_episodes = 10
while episode_counter < num_episodes:
    for agent in agents:
        agent_output = agent.step(time_step, is_evaluation=True)
        for output in agent_output:
            print(agent.player_id, output.action)
        time_step, rewards, dones, _ = env.step(agent_output, reset_if_done=True)
        print(dones)
    episode_counter += sum(dones)



0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 5
[True]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 5
[True]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 5
[True]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[False]
0 3
[False]
1 5
[True]
0 3
[False]
1 8
[False]
0 3
[False]
1 8
[Fal

In [ ]:
for ts in time_step:
    d = ts.observations['info_dict'][0]
    bt = int(d['bidder_type'].nonzero()[0])
    print(bt, straightforward.step(ts).action)
    print(d['sor_profits'])
    
    
    break

In [12]:
game.auction_params.all_bids

array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [1, 1],
       [1, 2],
       [2, 0],
       [2, 1],
       [2, 2]])

In [ ]:
np.array(game.auction_params.opening_prices)# @@ np.array([2,2])

In [ ]:
game.auction_params.max_budget

In [ ]:
game.auction_params.max_activity

In [ ]:
game.auction_params.activity

In [ ]:
# Action 7 is the way to go (straightforward)
# Actual played action is 